In [1]:
import random
import os
from datetime import datetime
from tqdm import tqdm
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder
import torch.nn as nn
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda', index=0)

In [2]:

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

In [3]:
train = pd.read_csv('../../DATA/train.csv')
test  = pd.read_csv('../../DATA/test.csv')

In [4]:
# 데이터 타입 변경, 열 이름 변경 

new_column_names = {
    'corporation': 'corp',
    'location': 'loc',
    'supply(kg)': 'supply',
    'price(원/kg)': 'price',
}

train = train.rename(columns=new_column_names)
test = test.rename(columns=new_column_names)

train['timestamp']  = pd.to_datetime(train['timestamp'])
test['timestamp']  = pd.to_datetime(test['timestamp'])

train['newitem'] = train['item'].str.cat([train['corp'], train['loc']], sep=' ')

In [5]:
#가격 시계열 price_data 생성하기 

def before_timeseries_data(train):
    unique_values = train['newitem'].unique()
    # 날짜 범위 생성
    start_date = datetime(2019, 1, 1).date()
    end_date =datetime(2023, 3, 3).date()
    dates = pd.date_range(start_date, end_date, freq='D')

    # 데이터프레임 생성
    price_data = pd.DataFrame(columns=['item'] + dates.strftime('%Y-%m-%d').tolist())
    price_data['item'] = unique_values

    # print(len(price_data)) 39 
    for i in range(len(price_data)) :
        price_data.iloc[i,1:] = train['price'][i*1523 : (i+1)*1523 ]
    price_data['item'] = price_data['item'].astype(str)
    price_data['corp'] = price_data['item'].str[3:4]

    price_data['loca'] = price_data['item'].str[-1:]
    price_data['product'] = price_data['item'].str[:2]
    price_data['product_loca'] =price_data['product'].str.cat(price_data['loca'], sep=' ')
    cols = price_data.columns[-4:]  # 가장 뒤의 3개 열의 열 이름을 선택
    price_data = price_data[cols.tolist() + price_data.columns[:-4].tolist()]  # 열 순서 변경
    
    return price_data

price_data = before_timeseries_data(train)

In [6]:


le = LabelEncoder()
price_data['product_loca'] = le.fit_transform(price_data['product_loca'])

/home/kiwoongyoon/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/home/kiwoongyoon/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):


In [7]:
price_df = price_data.copy()

In [50]:
# Class Definitions


class Model(nn.Module):
    def __init__(self, configs):
        super(Model, self).__init__()

        # remove this, the performance will be bad
        # self.lucky = nn.Embedding(configs.enc_in, configs.d_model // 2)

        # self.seq_len = configs.seq_len
        # self.pred_len = configs.pred_len
        # self.enc_in = configs.enc_in
        # self.patch_len = configs.patch_len
        # self.d_model = configs.d_model
        self.seq_len = 72
        self.pred_len = 28
        self.enc_in = 7
        self.patch_len = 16
        self.d_model = 256


        self.linear_patch = nn.Linear(self.patch_len, self.d_model)
        self.relu = nn.ReLU()

        self.gru = nn.GRU(
            input_size=self.d_model,
            hidden_size=self.d_model,
            num_layers=1,
            bias=True,
            batch_first=True,
        )

        self.pos_emb = nn.Parameter(torch.randn(self.pred_len // self.patch_len, self.d_model // 2))
        self.channel_emb = nn.Parameter(torch.randn(self.enc_in, self.d_model // 2))

        self.dropout = nn.Dropout(0.3)
        self.linear_patch_re = nn.Linear(self.d_model, self.patch_len)

    def forward(self, x):
        seq_last = x[:, -1:, :].detach()
        x = x - seq_last

        B, L, C = x.shape
        N = self.seq_len // self.patch_len
        M = self.pred_len // self.patch_len
        W = self.patch_len
        d = self.d_model

        xw = x.permute(0, 2, 1).reshape(B * C, N, -1)  # B, L, C -> B, C, L -> B * C, N, W
        xd = self.linear_patch(xw)  # B * C, N, W -> B * C, N, d
        enc_in = self.relu(xd)

        enc_out = self.gru(enc_in)[1].repeat(1, 1, M).view(1, -1, self.d_model) # 1, B * C, d -> 1, B * C, M * d -> 1, B * C * M, d

        dec_in = torch.cat([
            self.pos_emb.unsqueeze(0).repeat(B*C, 1, 1), # M, d//2 -> 1, M, d//2 -> B * C, M, d//2
            self.channel_emb.unsqueeze(1).repeat(B, M, 1) # C, d//2 -> C, 1, d//2 -> B * C, M, d//2
        ], dim=-1).flatten(0, 1).unsqueeze(1) # B * C, M, d -> B * C * M, d -> B * C * M, 1, d

        dec_out = self.gru(dec_in, enc_out)[0]  # B * C * M, 1, d

        yd = self.dropout(dec_out)
        yw = self.linear_patch_re(yd)  # B * C * M, 1, d -> B * C * M, 1, W
        y = yw.reshape(B, C, -1).permute(0, 2, 1) # B, C, H

        y = y + seq_last

        return y

class Data(Dataset):
    def __init__(self, X, Y):
        self.X = X
        self.Y = Y

    def __len__(self):
        return len(self.Y)
    
    def __getitem__(self, idx):
        return self.X[idx], self.Y[idx]

In [10]:
price_df.head(3)

,corp,loca,product,product_loca,item,2019-01-01,2019-01-02,2019-01-03,2019-01-04,2019-01-05,...,2023-02-22,2023-02-23,2023-02-24,2023-02-25,2023-02-26,2023-02-27,2023-02-28,2023-03-01,2023-03-02,2023-03-03
0,A,J,TG,8,TG A J,0.0,0.0,1728.0,1408.0,1250.0,...,2813.0,2770.0,2633.0,3155.0,0.0,2657.0,3922.0,3397.0,3195.0,3640.0
1,A,S,TG,9,TG A S,0.0,0.0,2526.0,2134.0,2075.0,...,3444.0,3481.0,3518.0,4201.0,0.0,4166.0,4009.0,4173.0,4219.0,4089.0
2,B,J,TG,8,TG B J,0.0,0.0,1692.0,1516.0,1471.0,...,4175.0,6216.0,3558.0,2412.0,0.0,3540.0,3141.0,6382.0,3558.0,3470.0


In [11]:
# Function to reshape the data into a time series format for each ID
def reshape_data(df):
    time_series_data = []
    for idx, row in df.iterrows():
        sales_data = row[5:].values.astype(float)
        time_series_data.append(sales_data)
    return np.array(time_series_data)

# Modified time_slide_df function to work with the current data format
def time_slide_df(data, window_size, forecast_size):
    data_list = []
    dap_list = []
    for idx in range(0, len(data) - window_size - forecast_size + 1):
        x = data[idx:idx + window_size].reshape(window_size, 1)
        y = data[idx + window_size:idx + window_size + forecast_size]
        data_list.append(x)
        dap_list.append(y)
    return np.array(data_list, dtype='float32'), np.array(dap_list, dtype='float32')

# Function to create DataLoader for each ID
def create_dataloader(data, window_size, forecast_size, batch_size):
    X, Y = time_slide_df(data, window_size, forecast_size)
    ds = Data(X, Y)
    return DataLoader(ds, batch_size=batch_size, shuffle=True)

# Reshape the data
time_series_data = reshape_data(price_df)

In [12]:
# Define the window size, forecast size, and batch size
window_size = 72   
forecast_size = 28
batch_size = 1024
epoch_count = 777
lr = 0.001
min_delta = 0.001
patience = 10

In [44]:
my_config = {
    'seq_len': 72,
    'pred_len': 28,
    'enc_in': 7,
    'patch_len': 16,
    'd_model': 256,
    'dropout': 0.3
}

#  self.seq_len = configs.seq_len
#         self.pred_len = configs.pred_len
#         self.enc_in = configs.enc_in
#         self.patch_len = configs.patch_len
#         self.d_model = configs.d_model


# Model Train 

In [51]:

future_predictions_by_id = {}
loss_history = {}
# Iterate through the data by ID
for idx, (id_val, data) in tqdm(enumerate(zip(price_df["item"], time_series_data)), total=len(price_df["item"])):
    # Standardizing the data
    mean_ = np.mean(data)
    std_ = np.std(data)
    
    # Check if std_ is zero and handle standardization accordingly
    if std_ == 0:
        standardized_data = data
    else:
        standardized_data = (data - mean_) / std_
        
    individual_loss_history = []
    
    # Create DataLoader
    train_dl = create_dataloader(standardized_data, window_size, forecast_size, batch_size)
    best_loss = float('inf')
    no_improvement_count = 0

    # Training the model
    DLinear_model = Model(my_config)
    DLinear_model.to(device)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(DLinear_model.parameters(), lr=lr)
    
    for epoch in range(1, epoch_count + 1):
        loss_list = []
        DLinear_model.train()
        for batch_idx, (data, target) in enumerate(train_dl):
            data, target = data.to(device), target.to(device)  # Move the data to the GPU if available
            optimizer.zero_grad()
            output = DLinear_model(data)
            loss = criterion(output, target.unsqueeze(-1))
            loss.backward()
            optimizer.step()
            loss_list.append(loss.item())
        
        if((epoch % 10) == 0):
            avg_loss = np.mean(loss_list)
            individual_loss_history.append(avg_loss)
            if avg_loss + min_delta < best_loss:
                best_loss = avg_loss
                no_improvement_count = 0
            else:
                no_improvement_count += 1
                if no_improvement_count >= patience:
                    break

    loss_history[id_val] = individual_loss_history

    # Predicting the future 15 days using the last window of data
    last_window_data = torch.tensor(standardized_data[-window_size:]).unsqueeze(0).unsqueeze(-1).float().to(device)
    future_prediction = DLinear_model(last_window_data)

    # Converting the prediction back to the original scale
    if std_ == 0:
        future_prediction = future_prediction.squeeze().detach().cpu().numpy()
    else:
        future_prediction = future_prediction.squeeze().detach().cpu().numpy() * std_ + mean_

    # Store the prediction
    future_predictions_by_id[id_val] = future_prediction
future_predictions_by_id

  0%|          | 0/39 [00:00<?, ?it/s]

RuntimeError: mat1 and mat2 shapes cannot be multiplied (4096x18 and 16x256)

In [28]:
(future_predictions_by_id.values)

<function dict.values>

In [32]:
submit = pd.read_csv('./../../DATA/sample_submission.csv')
submit

,ID,answer
0,TG_A_J_20230304,0
1,TG_A_J_20230305,0
2,TG_A_J_20230306,0
3,TG_A_J_20230307,0
4,TG_A_J_20230308,0
5,TG_A_J_20230309,0
6,TG_A_J_20230310,0
7,TG_A_J_20230311,0
8,TG_A_J_20230312,0
9,TG_A_J_20230313,0


In [41]:
findata = []
for id_val, predictions in future_predictions_by_id.items():
    rounded_predictions = np.round(predictions).astype(np.float64)
    findata.extend(rounded_predictions)

submit['answer'] = findata
submit.loc[submit['answer'] <= 0, 'answer'] = 0
    

In [44]:
submit.loc[submit.index % 7 == 1, 'answer'] = 0

In [46]:
submit.to_csv('../../DATA/SUBMIT/Dlinear1.csv', index=False)